<a href="https://colab.research.google.com/github/JishnuJayaraj/ML/blob/master/NLP/Bert/summarization/BertSummarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# SUMMARIZATION AND NER
!pip install transformers
# https://chriskhanhtran.github.io/posts/named-entity-recognition-with-transformers/

# 1. Summarization

In [ ]:
import pandas as pd 
import transformers
#from transformers import DistilBertTokenizer

In [ ]:
df = pd.read_csv('article_20.csv')

# name the columns
df.columns = ['rowno','date','heading','full_content','link','empty']

# cut down columns
articles = df[['heading','full_content']]
articles['word_count'] = articles['full_content'].apply(lambda x: len(str(x).split(" ")))
articles['content'] = articles['full_content'].str.slice(0,1024)
#articles['content4k'] = articles['full_content'].str.slice(0,4096)

articles.head()

In [ ]:
# Defining DistilBERT tokonizer
distil_bert = 'distilbert-base-uncased' # Pick any desired pre-trained model
roberta = 'roberta-base-uncase'

# change name here to change tokenizer
tokenizer = DistilBertTokenizer.from_pretrained(distil_bert, do_lower_case=True, add_special_tokens=True,
                                                max_length=512, pad_to_max_length=True)

In [ ]:
# Tokenize the document
def tokenize(sentences, tokenizer):
    input_ids, input_masks, input_segments = [],[],[]
    for sentence in tqdm(sentences):
        inputs = tokenizer.encode_plus(sentence, add_special_tokens=True, max_length=128, pad_to_max_length=True, 
                                             return_attention_mask=True, return_token_type_ids=True)
        input_ids.append(inputs['input_ids'])
        input_masks.append(inputs['attention_mask'])
        input_segments.append(inputs['token_type_ids'])        
        
    return np.asarray(input_ids, dtype='int32'), np.asarray(input_masks, dtype='int32'), np.asarray(input_segments, dtype='int32')

## Training
* Use Pretrained model directly as a classifier
* Transformer model to extract embedding and use it as input to another classifier.
* Fine-tuning a Pretrained transformer model on custom config and dataset.

In [ ]:
# SUMMARIZATION
from transformers import pipeline

#summarizer = pipeline("summarization")
summarizer = pipeline('summarization', model='bart-large-cnn', tokenizer='bart-large-cnn')

#from __future__ import print_function
#import ipywidgets as widgets

In [ ]:
#summarizer(article, max_length=90, min_length=20)
articles['summary'] = articles['content'].apply(summarizer)
#articles['summary'] = articles['content4k'].apply(summarizer)
#articles = articles[['heading','full_content','word_count','content4k','summary']]
#articles.to_csv('final.csv')
#print(summary[0]['summary_text'])

In [ ]:
# New Hugging face pipeline summarizer has a input text limit
# recreating a old summarizer
from transformers import BartForConditionalGeneration, BartTokenizer, AutoTokenizer, AutoModel
from typing import List

tokenizer = AutoTokenizer.from_pretrained("facebook/bart-large-cnn")
			
model = AutoModel.from_pretrained("facebook/bart-large-cnn")
   
def old_summarization_pipeline(text: List[str]) -> List[str]:
    #tokenizer = BartTokenizer.from_pretrained('bart-large-cnn')
    #model = BartForConditionalGeneration.from_pretrained('bart-large-cnn')
    input_ids = tokenizer.batch_encode_plus(text, return_tensors='pt', max_length=1024)['input_ids']
    summary_ids = model.generate(input_ids)
    summaries = [tokenizer.decode(s) for s in summary_ids]
    return summaries

# text = '=' * 10257  
text = "In a world where distributed product development and manufacturing are constantly gaining ground, collaboration capabilities are one of the factors that are key to success. The better the tools you have that can make extended enterprise collaboration easier, the more effectively the marketing, development and production work can be executed. What follows from this is broader collaborative flexibility, shorter time to market and other valuable advantages. Any of the big PLM systems have solutions for this, but just how effective are these collaboration tools in terms of a third-party non-engineering team involved in the product realization? These days, it’s not uncommon to work with teams and other people outside the engineering departments. Do they have access to the needed tools, such as those from the big PLM players Dassault Systémes, PTC or Siemens?"
old_summarization_pipeline(text)

In [ ]:
# NER

ner_tokenizer = pipeline("ner")
text = 'Jishnu is born in Kerala. He like to travel and eat alot. His favouite meal is chicken and soup. He works in Microsoft in California from this December'

#print(ner_tokenizer(art1))
print('************')
print(ner_tokenizer(text))

# BERT-NER from depend on defnition
[Mccormik fine tuning: Glue](https://mccormickml.com/2019/07/22/BERT-fine-tuning/)

[depends on def](https://www.depends-on-the-definition.com/named-entity-recognition-with-bert/)

[medium: painless fine Tuning](https://medium.com/swlh/painless-fine-tuning-of-bert-in-pytorch-b91c14912caa)